In [2]:
!which jupyter

/usr/local/bin/jupyter


In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from utils.imports import *

Using TensorFlow backend.


In [ ]:
shape = (61, 501, 501, 1)
def parser_train(serialized_example):
    features, sequence_features = tf.parse_single_sequence_example(
        serialized_example, context_features={
            'time_stamp': tf.FixedLenFeature([], tf.string),
        }, sequence_features={
            "raw_png": tf.FixedLenSequenceFeature([], dtype=tf.string)
        })
    
    x = tf.map_fn(tf.image.decode_png, sequence_features['raw_png'], dtype=tf.uint8,
                back_prop=False, swap_memory=False, infer_shape=False)
    x = tf.cast(x, tf.float32)
    x /= 255
    x.set_shape(shape)
    return x, features['time_stamp']

In [5]:
from utils.transforms import *

sz = 128
nt = 10
stats = np.fromfile('stat.csv', sep=',')

class Slice(Transform):
    """ Return a slice of the images
    
    Arguments:
    The same as the built-in function slice
    """
    def __init__(self, *args, **kwargs):
        self.slice = slice(*args, **kwargs)
        super().__init__(TfmType.NO)
        
    def do_transform(self, x, is_y):
        return x[self.slice]
    
aug_tfms = [Slice(nt)]
tfm, _ = tfms_from_stats(stats, sz, aug_tfms=aug_tfms, crop_type=CropType.NO)
tfm

[<utils.transforms.Scale object at 0x7fc90deec198>, <__main__.Slice object at 0x7fc90deec278>, <utils.transforms.Normalize object at 0x7fc90deec128>]

In [ ]:
bs = 3

filenames = tf.placeholder(tf.string, name='tfrecords')
training_filenames = '../data/tfrecords/train_1.tfrecords'
dataset = tf.data.TFRecordDataset(filenames)
dataset = dataset.map(parser_train).map(tfm).batch(bs).repeat()
# dataset = dataset.prefetch()
iterator = dataset.make_initializable_iterator()
iterator.initializer.run({filenames: training_filenames})
x, time_stamp = iterator.get_next()

In [ ]:
input_fn(fn_pattern, mode=tf.esti)